#Гугл диск


In [50]:
#from google.colab import drive
#drive.mount('/content/drive')

#Нейросеть

In [51]:
import os, shutil
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten 
from tensorflow.keras.applications import ResNet152V2
from keras.preprocessing.image import ImageDataGenerator, load_img

##Константы

In [52]:
FAST_RUN = False
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)
# определим batch_size
batch_size=15

##Загружаем предварительно обученную нейронную сеть ResNet152V2
Создадим экземпляр модели ResNet152V2

In [53]:
ResNet152V2_net = ResNet152V2(weights='imagenet', 
                  include_top=False, 
                  input_shape=input_shape)

In [54]:
ResNet152V2_net.trainable = False

## Классификатор

In [55]:
model = Sequential()
# добавляем в модель сеть ResNet152V2 в качестве слоя
model.add(ResNet152V2_net)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))

In [56]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152v2 (Model)          (None, 4, 4, 2048)        58331648  
_________________________________________________________________
flatten_2 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               8388864   
_________________________________________________________________
dense_7 (Dense)              (None, 512)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 9)                 4617      
Total params: 66,856,713
Trainable params: 8,525,065
Non-trainable params: 58,331,648
__________________________________

In [57]:
model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])

##Веса модели


In [58]:
# Веса модели
# Файл model-cnn-S2.h5
# Укажите путь к файлу
model.load_weights("/content/drive/My Drive/Sharing-Economy-neural-network/model-cnn-S2.h5") 
#model.load_weights("/content/model-cnn-S2.h5") 



#Классификация


##Распределение по категориям

In [59]:
!mkdir imgs

mkdir: cannot create directory ‘imgs’: File exists


In [60]:
def distribution_by_category():
    test_filenames = os.listdir("/content/imgs/")
    test_df_img = pd.DataFrame({
        'filename': test_filenames
    })
    nb_samples = test_df_img.shape[0]
    test_df_img.head() 



    test_gen = ImageDataGenerator(rescale=1./255)
    test_generator = test_gen.flow_from_dataframe(
        test_df_img, 
        "/content/imgs", 
        x_col='filename',
        y_col=None,
        class_mode=None,
        target_size=IMAGE_SIZE,
        batch_size=batch_size,
        shuffle=False
    )


    predict = model.predict(test_generator, steps=np.ceil(nb_samples/batch_size))

    category_dict = {0:'Готовые блюда', 1:'Консервы', 2:'Крупы, макаронные изделия, чай и приправы', 
                      3:'Молочная продукция', 4:'Мясное, рыба', 5:'Овощи, фрукты, орехи', 6:'Реклама', 
                      7:'Соки, напитки', 8:'Хлебобулочные и кондитерские изделия'}     
    predict_index = np.argmax(predict, axis=-1)
    for index in predict_index:
      print('Нейросеть Категория:', category_dict[index])
    return predict_index


#vk_parser

In [61]:
import requests
import json
import time

In [62]:
version = '5.110'
token = '06f914db06f914db06f914db67068b6414006f906f914db58235e27d29376af1fa9f903'

###img_processing

In [63]:
def vk_respons(domain,token,version,count = 1,offset = 1):
    response = requests.get('https://api.vk.com/method/wall.get',
                            params={
                                'access_token': token,
                                'v': version,
                                'domain': domain,
                                'count': count,
                                'offset': offset
                            }
                            )
    vk_answer = response.json()
    return vk_answer

def downlaod_img(img,path):
    p = requests.get(img)
    out = open(path, "wb")
    out.write(p.content)
    out.close()


def tests_preparing(groups):
    for i in range(len(groups)):
        domain = groups[i]
        count = 100
        vk_answer = vk_respons(domain,token,version,count)
        for j in range(vk_answer['response']['items'].__len__()):
            post_id = vk_answer['response']['items'][j]['id']
            owner_id = vk_answer['response']['items'][j]['owner_id']
            if 'attachments' in vk_answer['response']['items'][j].keys():
                for n in range(vk_answer['response']['items'][i]['attachments'].__len__()):
                    try:
                        if 'photo' in vk_answer['response']['items'][j]['attachments'][n].keys():
                            path = '../test_imgs/'+str(owner_id)+'_'+str(post_id)+'('+str(n)+').jpg'
                            url = vk_answer['response']['items'][j]['attachments'][n]['photo']['sizes'][2]['url']
                            downlaod_img(url, path)
                    except:
                        pass




###parser

In [64]:
# https://nanib0ots.pythonanywhere.com


def write_json(post_id, owner_id):
    post_id = str(post_id)
    owner_id = str(owner_id)
    flag = False
    try:
        json_d = json.load(open('groups.json'))
    except:
        json_d = {}
    if owner_id in json_d.keys():
        if json_d[owner_id] != post_id:
            flag = True
            json_d[owner_id] = post_id
            with open('groups.json', 'w') as file:
                json.dump(json_d, file, indent=2, ensure_ascii=False)
    else:
        json_d[owner_id] = post_id
        flag = True
        with open('groups.json', 'w') as file:
            json.dump(json_d, file, indent=2, ensure_ascii=False)
    return flag


def vk_respons(domain, token, version, count=1, offset=1):
    response = requests.get('https://api.vk.com/method/wall.get',
                            params={
                                'access_token': token,
                                'v': version,
                                'domain': domain,
                                'count': count,
                                'offset': offset
                            }
                            )
    vk_answer = response.json()
    return vk_answer


def vk_pars_func(data):
    for i in range(len(data)):
        domain = data[i]['group_id']
        cities = list(set(data[i]['city']))

        vk_answer = vk_respons(domain, token, version)
        post_id = vk_answer['response']['items'][0]['id']
        owner_id = vk_answer['response']['items'][0]['owner_id']
        writer = write_json(post_id, owner_id)
        if writer:
            print('Пост Вк:',
                  'link:', 'https://vk.com/' + domain + '?w=wall' + str(owner_id) + '_' + str(post_id), '\n')
            item = vk_answer['response']['items'][0]

            text = item['text']


            # Определение города
            if len(cities) == 1:
                city = [cities[0]]
            else:
                city = cities
                for item_city in cities:
                    if item_city.lower() in text.lower():
                       city = [item_city]
                       break



            # Определение категории
            neural_network = False
            if 'attachments' in item.keys():
                attachments = item['attachments']
                n = 0 # индекс изображени
                for att in attachments:
                    neural_network = True
                    # Проверяем есть ли во вложениях фото
                    if att['type'] == 'photo':
                        path = './imgs/' + str(owner_id) + '_' + str(post_id) + '(' + str(n) + ').jpg'
                        url = att['photo']['sizes'][2]['url']
                        n += 1
                        downlaod_img(url, path) # загрузка изображений
            
            # Нейросеть 
            category = [] 
            folder = './imgs/'
            if neural_network:          
                category_dict = {0:'Готовые блюда', 1:'Консервы', 2:'Крупы, макаронные изделия, чай и приправы', 
                      3:'Молочная продукция', 4:'Мясное, рыба', 5:'Овощи, фрукты, орехи', 6:'Реклама', 
                      7:'Соки, напитки', 8:'Хлебобулочные и кондитерские изделия'} 
                try:              
                    predict_index = distribution_by_category()
                    if not 6 in predict_index:
                        for index in predict_index:
                            category.append(category_dict[index])
                except Exception as e: 
                    print(e) 

            # отчищаем папку с загрузками
            folder = './imgs/' 
            for the_file in os.listdir(folder): 
                file_path = os.path.join(folder, the_file) 
                try: 
                    if os.path.isfile(file_path): 
                        os.unlink(file_path) 
                    elif os.path.isdir(file_path): 
                        shutil.rmtree(file_path) 
                except Exception as e: 
                    print(e) 

                  

            #  Отправка поста боту
            send = {'link': 'https://vk.com/' + domain + '?w=wall' + str(owner_id) + '_' + str(post_id),
                    'category': category, 'city': city, 'text': text}
            bot_send = json.dumps(send)
            print('Сообщение боту:\n',bot_send, '\n')
            requests.get('https://nanib0ots.pythonanywhere.com/bot/post', send, cookies={'parser_key': '12345678'})
            print('=========================================')


def main():
    while (True):
        try:
            response = requests.get('https://nanib0ots.pythonanywhere.com/bot/get_data', cookies={'parser_key': '12345678'})
            if response.status_code != 200:
                time.sleep(5)
                main()
            data = json.loads(response.content)
            print('=========================================')
            print('Ответ от базы данных\n', data,'\n')
        except requests.exceptions.ConnectionError:
            print('ConnectionError')
            time.sleep(40)
            main()
        vk_pars_func(data)
        time.sleep(40)


if __name__ == '__main__':
    main()


Ответ от базы данных
 [{'title': 'Фудшеринг I Отдам даром еду I Челябинск', 'city': ['Челябинск'], 'group_id': 'fudshering74'}, {'title': 'Хелпфуд. Фудшеринг. Санкт-Петербург', 'city': ['СПБ'], 'group_id': 'club_helpfoodspb'}, {'title': 'Фудшеринг Отдам даром еду', 'city': ['Москва', 'СПБ'], 'group_id': 'sharingfood'}, {'title': 'Фудшеринг Отдам даром в Иркутске', 'city': ['Иркутск'], 'group_id': 'sharingfood_irk'}, {'title': 'Наша группа', 'city': ['Иркутск'], 'group_id': 'NaniB0ots_food'}] 

Пост Вк: link: https://vk.com/fudshering74?w=wall-101826852_6839 

Found 1 validated image filenames.
Нейросеть Категория: Мясное, рыба
Сообщение боту:
 {"link": "https://vk.com/fudshering74?w=wall-101826852_6839", "category": ["\u041c\u044f\u0441\u043d\u043e\u0435, \u0440\u044b\u0431\u0430"], "city": ["\u0427\u0435\u043b\u044f\u0431\u0438\u043d\u0441\u043a"], "text": "[id427437472|\u039c\u0430\u0440\u0438\u044f]"} 

Пост Вк: link: https://vk.com/club_helpfoodspb?w=wall-161997638_10348 

Found 1 

KeyboardInterrupt: ignored

In [ ]:
#!rm ./imgs/*.jpg
!rm ./groups.json